In [ ]:
# # 필요한 라이브러리 설치하기
# # 주피터 노트북에서 실행하지 마시고, 가급적이면 명령어 프롬프트에서 실행해 주세요.
# !conda create -n rag python=3.10
# !conda install openai langchain chromadb streamlit python-dotenv jupyter -q -y
# !pip install sentence_transformers unstructured -q
# !pip install "unstructured[pdf]" -q

## 1. 디렉토리에 있는 문서들을 적재하기

In [1]:
# 해당 디렉토리에 있는 PDF 문서들을 적재한다.
from langchain.document_loaders import DirectoryLoader

directory = './pdf'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

1

## 2. 문서내용을 청크단위로 오버랩해서 분리하기
- <span style="font-size:14px">긴 문서를 한 번에 처리하는 것은 문맥을 제대로 보존하기 어려워 문장이나 청크로 나누어 처리하면 각 청크(chunk)의 문맥을 보존하면서 모델에 입력할 수 있음</span>
- <span style="font-size:14px">청크 간에 일부 텍스트를 중복하여 처리함으로써 정보 손실을 줄일 수 있음(overlap)</span><br>
<img src="./images/chunks.png" alt="이미지 예시" width="55%" style="float: left; margin: 0px;"><br>

In [2]:
# 해당 문서내용을 청크단위(1,000)로 오버랩핑(20)해 가면서 분리하는 작업을 한다.
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

6


## 3. 문서내용을 인베딩벡터로 변환형 Chroma DB로 생성하기
- <span style="font-size:14px">벡터 임베딩은 텍스트, 이미지, 오디오 등과 같은 숫자가 아닌 데이터의 숫자 표현(숫자 벡터)임</span>
- <span style="font-size:14px">벡터 저장소는 벡터 임베딩을 컬렉션 형태로 저장하는 데 사용되는 데이터베이스임</span>
- <span style="font-size:14px"><b>Chroma DB</b>는 벡터 임베딩 데이터에서 정보를 효율적으로 저장하고 검색을 지원함</span>

<img src="./images/chroma-db.png" alt="이미지 예시" width="85%" style="float: left; margin: 0px;">

- 참고자료: https://zephyrnet.com/ko/%EC%83%9D%EC%84%B1-AI%EB%A5%BC-%EC%9C%84%ED%95%9C-%EB%B2%A1%ED%84%B0-%EC%A0%80%EC%9E%A5%EC%86%8C%EC%9D%B8-Chroma-db-%EA%B0%80%EC%9D%B4%EB%93%9C/

In [3]:
# 해당 문서를 인베딩벡터로 변환하여 Chroma DB를 생성한다. 
# (임시조치) 
# 버전문제인지 실행하면 오류가 발생하는데, 다시 한번 실행해 주면 오류가 발생하지 않고 정상 동작한다. 
# 오류내용: InvalidInputException: Invalid Input Error: Required module 'pandas.core.arrays.arrow.dtype' failed to import, due to the following Python exception
# (조치내용)
# 2023.09.09 - 가상화환경을 기존 Python 3.9 -> 3.10로 업그레이드 시 해결됨
# from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = HuggingFaceEmbeddings()

db = Chroma.from_documents(docs, embeddings)

## 4. 테스트 해보기

In [2]:
# 유사도가 비슷한 문서를 가져오는지 테스트를 해본다.
query = "Mackbook Pro M2은 언제 출시되었어?"
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = HuggingFaceEmbeddings()

persist_directory = "./chroma_db"
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
matching_docs = db.similarity_search(query)

matching_docs

[Document(page_content='4. K v 는 A 가중치를 적용한 사운드 파워 수준의 상한을 계산하기 위한 통계\n\n가산기입니다.\n\n5. 수량을 의미하는 L W A,c(이전에는 L W Ad라고 했음)는 L W A,m과 K v 의\n\n합으로 계산될 수 있습니다.\n\n6. 무선 웹 테스트에서는 미디어를 포함하는 혼합된 콘텐츠 탭 6개를\n\n탐색합니다.\n\n7. 구성 테스트됨: M2, 8코어 CPU, 10코어 GPU, 8GB 통합메모리,\n\n512GB 저장 장치.\n\n1. 테스트는 2022년 5월 Apple에서 Apple M2, 8코어 CPU, 10코어\n\nGPU, 8GB RAM 및 256GB SSD를 탑재한 MacBook Pro 13 시제품을\n\n사용해 진행했습니다. 무선 웹 테스트는 디스플레이의 밝기를 제일\n\n어두운 상태로부터 8단계 밝게 한 상태에서 무선으로 인기 웹사이트\n\n25곳을 방문하는 방식으로 배터리 사용 시간을 측정했습니다. Apple\n\nTV 앱 동영상 재생 테스트는 디스플레이의 밝기를 제일 어두운\n\n상태로부터 8단계 밝게 한 상태에서 HD 1080p 콘텐츠를 재생하는\n\n방식으로 배터리 사용 시간을 측정했습니다. 배터리 사용 시간은 사용\n\n패턴 및 설정에 따라 다를 수 있습니다. 자세한\n\n내용은 apple.com/kr/batteries를 참고하십시오.\n\n2. 1GB = 10억 바이트, 1TB = 1조 바이트입니다. 실제 포맷된 용량은 더\n\n적습니다.\n\n3. 무게는 구성 및 제조 과정에 따라 다를 수 있습니다.\n\n4. iMovie, GarageBand, Pages, Numbers, Keynote는 Mac App\n\nStore에서 다운로드할 수 있습니다. 앱을 다운로드하려면 Apple ID\n\n그리고 각 앱에 필요한 OS 버전과 호환되는 기기가 필요합니다.\n\n5. 제품 출시 당시를 기준으로 한 데이터입니다.\n\n6. ENERGY STAR 및 ENERGY STAR 마크는 미국 

## 5. Chroma DB를 디스크에 저장하기
<img src="./images/chroma-db-disk.png" alt="이미지 예시" width="55%" style="float: left; margin: 0px;"><br>

In [5]:
# 생성된 Chroma DB를 디스크에 저장한다.
persist_directory = "chroma_db"

vectordb = Chroma.from_documents(
    documents=docs, embedding=embeddings, persist_directory=persist_directory
)

vectordb.persist()

## 6. 테스트 해보기 - ChatGPT with RAG
<img src="./images/query-process.png" alt="이미지 예시" width="65%" style="float: left; margin: 0px;">

In [4]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

load_dotenv()
openai_token = os.environ.get("OPENAI_TOKEN")
os.environ["OPENAI_API_KEY"] = openai_token

model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)

chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

query = "Mackbook Pro M2은 언제 출시되었어?"
matching_docs = db.similarity_search(query)
answer =  chain.run(input_documents=matching_docs, question=query)
answer



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
4. K v 는 A 가중치를 적용한 사운드 파워 수준의 상한을 계산하기 위한 통계

가산기입니다.

5. 수량을 의미하는 L W A,c(이전에는 L W Ad라고 했음)는 L W A,m과 K v 의

합으로 계산될 수 있습니다.

6. 무선 웹 테스트에서는 미디어를 포함하는 혼합된 콘텐츠 탭 6개를

탐색합니다.

7. 구성 테스트됨: M2, 8코어 CPU, 10코어 GPU, 8GB 통합메모리,

512GB 저장 장치.

1. 테스트는 2022년 5월 Apple에서 Apple M2, 8코어 CPU, 10코어

GPU, 8GB RAM 및 256GB SSD를 탑재한 MacBook Pro 13 시제품을

사용해 진행했습니다. 무선 웹 테스트는 디스플레이의 밝기를 제일

어두운 상태로부터 8단계 밝게 한 상태에서 무선으로 인기 웹사이트

25곳을 방문하는 방식으로 배터리 사용 시간을 측정했습니다. Apple

TV 앱 동영상 재생 테스트는 디스플레이의 밝기를 제일 어두운

상태로부터 8단계 밝게 한 상태에서 HD 1080p 콘텐츠를 재생하는

방식으로 배터리 사용 시간을 측정했습니다. 배터리 사용 시간은 사용

패턴 및 설정에 따라 다를 수 있습니다. 자세한

내용은 apple.com/kr/batteries를 참고하십시오.

2. 1GB = 10억 바이트, 1TB = 1조 바이트입니다. 실제 포맷된 용량은 더

적습니다.

3. 무게는 구성 및 제조 과정에 따

'MacBook Pro M2는 2022년에 출시되었습니다.'

In [11]:
from langchain.chains import RetrievalQA

retrieval_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=db.as_retriever())
retrieval_chain.run(query)

'MacBook Pro M2은 2022년에 출시되었습니다.'

In [3]:
import os
import shutil

# Chroma DB를 해당 디렉토리에 저장하기 전에 이전 데이터를 삭제한다.
persist_directory = "./chroma_db"
if os.path.exists(persist_directory):
    shutil.rmtree(persist_directory)